# Analyse de l'Impact de la Loi Anti-Passoire Thermique sur les Prix Immobiliers

Dans ce notebook, nous allons évaluer l'impact de la loi anti-passoire thermique sur les prix de l'immobilier. Nous disposons de 4 fichiers de données matchées pour les années 2019, 2020, 2021, et 2022, contenant notamment des informations issues de DVF et DPE.

L'analyse se fera en deux temps :
1. Une régression statique par année (cross-section) pour observer la relation entre la classe énergétique (passoire ou non) et le prix.
2. Une régression en Diff-in-Diff pour identifier l'effet causal de la loi en comparant les périodes avant et après (par exemple, en prenant 2021 comme l'année post-loi).

Nous allons détailler chaque étape ci-dessous.


In [1]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf


## Chargement des Données

Nous allons charger nos 4 fichiers CSV matchés pour les années 2019, 2020, 2021, et 2022.  
Assure-toi que les fichiers (par exemple `final2019.csv`, `final2020.csv`, etc.) se trouvent dans le même dossier que ce notebook, ou adapte les chemins en conséquence.


In [5]:
# Charger les 4 fichiers CSV
df_2019 = pd.read_csv('final2019.csv')
df_2020 = pd.read_csv('final2020.csv')
df_2021 = pd.read_csv('final2019.csv')
df_2022 = pd.read_csv('final2022.csv')
df_2023 = pd.read_csv('final2023.csv')

# Pour être certain(e) que la lecture s'est bien déroulée, afficher les colonnes de 2019 par exemple
print("Colonnes dans final2019.csv :", df_2019.columns.tolist())


/tmp/ipykernel_45595/2835024718.py:2: DtypeWarning: Columns (7,8,20,22,24,26,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2019 = pd.read_csv('final2019.csv')
/tmp/ipykernel_45595/2835024718.py:3: DtypeWarning: Columns (7,8,20,22,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020 = pd.read_csv('final2020.csv')
/tmp/ipykernel_45595/2835024718.py:4: DtypeWarning: Columns (7,8,20,22,24,26,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2021 = pd.read_csv('final2019.csv')
/tmp/ipykernel_45595/2835024718.py:5: DtypeWarning: Columns (7,8,20,22,24,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022 = pd.read_csv('final2022.csv')


Colonnes dans final2019.csv : ['Date_établissement_DPE', 'Etiquette_DPE', 'Type_bâtiment', 'Année_construction', 'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)', 'N°_département_(BAN)', 'Code_INSEE_(BAN)', 'Adresse_Normalisee', 'id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'nature_culture_speciale', 'surface_terrain', 'longi

/tmp/ipykernel_45595/2835024718.py:6: DtypeWarning: Columns (7,8,20,22,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023 = pd.read_csv('final2023.csv')


In [6]:
# Ajouter la variable 'transaction_year' pour le fichier 2019
df_2019['transaction_year'] = 2019
df_2020['transaction_year'] = 2020
df_2021['transaction_year'] = 2021
df_2022['transaction_year'] = 2022
df_2023['transaction_year'] = 2023

# Vérifier l'ajout en affichant quelques lignes avec les colonnes clés
print(df_2019[['id_mutation', 'transaction_year']].head())


   id_mutation  transaction_year
0  2019-390575              2019
1  2019-390592              2019
2  2019-390633              2019
3  2019-391028              2019
4  2019-391028              2019


## Concaténation des Données

Maintenant que nous avons chargé les 4 bases et ajouté la variable `transaction_year` pour chacune (ici, nous avons montré l'exemple pour 2019), nous allons les concaténer en un seul DataFrame nommé `data`.  
Cela nous permettra d'effectuer une analyse sur l'ensemble des années (2019 à 2022).


In [7]:
# Concaténer les DataFrames des différentes années
data = pd.concat([df_2019, df_2020, df_2021, df_2022], ignore_index=True)

# Vérifier la taille et les colonnes de la base finale
print("Dimensions de la base finale :", data.shape)
print("Liste des colonnes disponibles :", data.columns.tolist())

# Afficher les premières lignes pour confirmer
print(data.head())


Dimensions de la base finale : (365535, 52)
Liste des colonnes disponibles : ['Date_établissement_DPE', 'Etiquette_DPE', 'Type_bâtiment', 'Année_construction', 'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)', 'N°_département_(BAN)', 'Code_INSEE_(BAN)', 'Adresse_Normalisee', 'id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'natu